## Using ADQL to download Gaia data

This is the first in a series of lessons related to astronomy data.  As a running example, we will replicate part of the analysis in a recent paper, "[Off the beaten path: Gaia reveals GD-1 stars outside of the main stream](https://arxiv.org/abs/1805.00425)" by Adrian M. Price-Whelan and Ana Bonaca.

As the abstract explains, "Using data from the Gaia second data release combined with Pan-STARRS photometry, we present a sample of highly-probable members of the longest cold stream in the Milky Way, GD-1."

GD-1 is a [stellar stream](https://en.wikipedia.org/wiki/List_of_stellar_streams) which is "an association of stars orbiting a galaxy that was once a globular cluster or dwarf galaxy that has now been torn apart and stretched out along its orbit by tidal forces."

The two datasets used in this study are
 
* [Gaia](https://en.wikipedia.org/wiki/Gaia_(spacecraft)), which is "a space observatory of the European Space Agency (ESA), launched in 2013 ... designed for astrometry: measuring the positions, distances and motions of stars with unprecedented precision", and

* [PanSTARRS](https://en.wikipedia.org/wiki/Pan-STARRS): "The Panoramic Survey Telescope and Rapid Response System, located at Haleakala Observatory, Hawaii, US, consists of astronomical cameras, telescopes and a computing facility that is surveying the sky for moving or variable objects on a continual basis, and also producing accurate astrometry and photometry of already-detected objects."

Both of these datasets are very large, which can make them challenging to work with.  One of the goals of this workshop is to provide tools for working with large datasets.

One of the most important of those tools is a "query language", which is a way to query a large database and efficiently select the information you need.  So that's where we'll start.

The query language we'll use is ADQL, which stands for "Astronomical Data Query Language".

ADQL is a dialect of [SQL](https://en.wikipedia.org/wiki/SQL) (Structured Query Language), which is by far the most commonly used query language.  Almost everything you learn about ADQL also works in SQL.

[The reference manual for ADQL is here](http://www.ivoa.net/documents/ADQL/20180112/PR-ADQL-2.1-20180112.html).

But you might find it easier to learn from [this ADQL Cookbook](https://www.gaia.ac.uk/data/gaia-data-release-1/adql-cookbook).

## Getting Gaia data

The library we'll use to get Gaia data is [Astroquery](https://astroquery.readthedocs.io/en/latest/).  If you are running this notebook on your own computer, you might have to install Astroquery.  You should have received instructions for this before the workshop.

If you are running this notebook on Colab, you can run the following cell to install Astroquery and a couple of other libraries we'll use.

In [1]:
# If we're running on Colab, install libraries

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery astro-gala pyia
    !mkdir data

From Astroquery we can import `Gaia`, which is an [object that represents a connection to the Gaia database](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html).

Running this import statement has the effect of creating a [TAP+](http://www.ivoa.net/documents/TAP/) connection; TAP stands for "Table Access Protocol".  It is a network protocol for sending queries to the database and getting back the results.

In [2]:
from astroquery.gaia import Gaia

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


I'm not sure why it seems to create two connections.

What is a database, anyway?  Most generally, it can be any collection of data, but when we are talking about ADQL or SQL:

* A database is a collection of one or more named tables.

* Each table is a 2-D array with one or more named columns of data.

We can use `load_tables` to get the names of the tables in the Gaia database.  With the option `only_names=True`, it loads information about the tables, the "metadata", not the data itself.

In [3]:
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print(table.get_qualified_name())

INFO: Retrieving tables... [astroquery.utils.tap.core]
INFO: Parsing tables... [astroquery.utils.tap.core]
INFO: Done. [astroquery.utils.tap.core]
external.external.apassdr9
external.external.gaiadr2_geometric_distance
external.external.galex_ais
external.external.ravedr5_com
external.external.ravedr5_dr5
external.external.ravedr5_gra
external.external.ravedr5_on
external.external.sdssdr13_photoprimary
external.external.skymapperdr1_master
external.external.tmass_xsc
public.public.hipparcos
public.public.hipparcos_newreduction
public.public.hubble_sc
public.public.igsl_source
public.public.igsl_source_catalog_ids
public.public.tycho2
public.public.dual
tap_config.tap_config.coord_sys
tap_config.tap_config.properties
tap_schema.tap_schema.columns
tap_schema.tap_schema.key_columns
tap_schema.tap_schema.keys
tap_schema.tap_schema.schemas
tap_schema.tap_schema.tables
gaiadr1.gaiadr1.aux_qso_icrf2_match
gaiadr1.gaiadr1.ext_phot_zero_point
gaiadr1.gaiadr1.allwise_best_neighbour
gaiadr1.gaiad

So that's a lot of tables.  The ones we'll use are:

* gaiadr2.gaia_source, which contains Gaia data from [data release 2](https://www.cosmos.esa.int/web/gaia/data-release-2),

* gaiadr2.panstarrs1_original_valid, which contains the photometry data we'll use from PanSTARRS, and

* gaiadr2.panstarrs1_best_neighbour, which we'll use to cross-match each star observed by Gaia with the same star observed by PanSTARRS.

We can use `load_table` (not `load_tables`) to get the metadata for a single table.  The name of this function is misleading, because it only downloads metadata. 

In [4]:
table = Gaia.load_table('gaiadr2.gaia_source')
table

Retrieving table 'gaiadr2.gaia_source'
Parsing table 'gaiadr2.gaia_source'...
Done.


Notice one gotcha: in the list of table names, this table appears as `gaiadr2.gaiadr2.gaia_source`, but when we load the metadata, we refer to it as `gaiadr2.gaia_source`.

Jupyter shows that the result is an object of type `TapTableMeta`, but it does not display the contents.

To see the metadata, we have to print the object.

In [5]:
print(table)

TAP Table name: gaiadr2.gaiadr2.gaia_source
Description: This table has an entry for every Gaia observed source as listed in the
Main Database accumulating catalogue version from which the catalogue
release has been generated. It contains the basic source parameters,
that is only final data (no epoch data) and no spectra (neither final
nor epoch).
Num. columns: 96


The following loop prints the names of the columns in the table.

In [6]:
for column in (table.columns):
    print(column.name)

solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pmra_pmdec_corr
astrometric_n_obs_al
astrometric_n_obs_ac
astrometric_n_good_obs_al
astrometric_n_bad_obs_al
astrometric_gof_al
astrometric_chi2_al
astrometric_excess_noise
astrometric_excess_noise_sig
astrometric_params_solved
astrometric_primary_flag
astrometric_weight_al
astrometric_pseudo_colour
astrometric_pseudo_colour_error
mean_varpi_factor_al
astrometric_matched_observations
visibility_periods_used
astrometric_sigma5d_max
frame_rotator_object_type
matched_observations
duplicated_source
phot_g_n_obs
phot_g_mean_flux
phot_g_mean_flux_error
phot_g_mean_flux_over_error
phot_g_mean_mag
phot_bp_n_obs
phot_bp_mean_flux
phot_bp_mean_flux_error
phot_bp_mean_flux_over_error
phot_bp_mean_ma

To find out what the columns mean, you can read [the documentation of this table here](https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html).

## Writing queries

Now you might be wondering how we actually download the data.  With tables this big, you generally don't.  Instead, you use queries to select only the data you want.

To do that, we're going to write an ADQL query.  Here's an example:

In [7]:
query1 = """SELECT 
TOP 10
source_id, ref_epoch, ra, dec, parallax 
FROM gaiadr2.gaia_source"""

The words in uppercase are ADQL keywords:

* `SELECT` indicates that we are selecting data (as opposed to adding or modifying data).

* `TOP` indicates that we only want the first 10 rows of the table, which is useful for testing a query before asking for all of the data.

* `FROM` specifies which table we want data from.

The third line is a list of column names, indicating which columns we want.  

I wrote the column names in lowercase to make it clear that they are not keywords.  This is a common style, but it is not required.

To run this query, we use `Gaia.launch_job`:

In [8]:
job1 = Gaia.launch_job(query1)
job1

The result is an object that represents the job running on a Gaia server.

If you print it, it displays metadata for the forthcoming table.

In [9]:
print(job1)

<Table length=10>
   name    dtype  unit                            description                             n_bad
--------- ------- ---- ------------------------------------------------------------------ -----
source_id   int64      Unique source identifier (unique within a particular Data Release)     0
ref_epoch float64   yr                                                    Reference epoch     0
       ra float64  deg                                                    Right ascension     0
      dec float64  deg                                                        Declination     0
 parallax float64  mas                                                           Parallax     1
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20200728114730.xml.gz
Results: None


Now we can get the results:


In [10]:
results1 = job1.get_results()
type(results1)

astropy.table.table.Table

The results is an [Astropy Table](https://docs.astropy.org/en/stable/table/) is similar to a table in an SQL database except:

* SQL databases are stored on disk drives, so they are persistent; that is, they "survive" even if you turn off the computer.  An Astropy `Table` is stored in memory; it disappears when you turn off the computer (or shut down this Jupyter notebook).

* SQL databases are designed to process queries.  And Astropy `Table` can perform some query-like operations, like selecting columns and rows.  But these operations use Python syntax, not SQL.

Jupyter knows how to display the contents of a `Table`.

In [11]:
results1

source_id,ref_epoch,ra,dec,parallax
,yr,deg,deg,mas
int64,float64,float64,float64,float64
5235310562546550400,2015.5,170.5618633106856,-67.46135363548994,0.5526448406265922
5235297235264005248,2015.5,170.81275310150227,-67.76467918585801,-0.008364953885495475
5235307714984673024,2015.5,170.74885441056904,-67.5709447855931,0.2843484392405459
5235320909100506752,2015.5,170.4194665931764,-67.47560971714014,0.07347483811378189
5235312628401884672,2015.5,170.12241187533036,-67.67525520385453,0.1708950029591145
5235320221929294208,2015.5,170.40826902057344,-67.55270731892348,--
5235314621267827840,2015.5,170.3029312675676,-67.54344977691999,0.17536376628442685
5235320084490908672,2015.5,169.80194700826704,-67.41538251912358,0.5827214577626764


Each column has a name, units, and a data type.  

For example, the units of `ra` and `dec` are degrees, and their data type is `float64`, which is a 64-bit floating-point number, used to store measurements with a fraction part.

**Exercise:** Read [the documentation of this table](https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html) and select at least one column name that looks interesting to you.  Add the columns you selected to the query and run it again.  What are the units of the column you selected?  What is its data type?

## Asynchronous queries

`launch_job` asks the server to run the job "synchronously", which normally means it runs immediately.  But synchronous jobs are limited to 2000 rows.  For queries that return more rows, you have to run "asynchronously", which mean they might take longer to get started.

The results of an asynchronous query are stored in a file on the server, so you can start a query and come back later to get the results.

For anonymous users, files are kept for three days.

We could run the same query asynchronously, but to live things up, let's add a new keyword, `WHERE`:

In [12]:
query2 = """SELECT 
TOP 100
source_id, ref_epoch, ra, dec, parallax
FROM gaiadr2.gaia_source
WHERE parallax < 1
"""

A `WHERE` clause indicates which rows we want; in this case, the query selects only rows "where" `parallax` is less than 1.

We use `launch_job_async` to submit an asynchronous query.

In [13]:
job2 = Gaia.launch_job_async(query2)
print(job2)

INFO: Query finished. [astroquery.utils.tap.core]
<Table length=100>
   name    dtype  unit                            description                            
--------- ------- ---- ------------------------------------------------------------------
source_id   int64      Unique source identifier (unique within a particular Data Release)
ref_epoch float64   yr                                                    Reference epoch
       ra float64  deg                                                    Right ascension
      dec float64  deg                                                        Declination
 parallax float64  mas                                                           Parallax
Jobid: 1595951250672O
Phase: COMPLETED
Owner: None
Output file: async_20200728114730.vot
Results: None


And here are the results.

In [14]:
results2 = job2.get_results()
results2

source_id,ref_epoch,ra,dec,parallax
,yr,deg,deg,mas
int64,float64,float64,float64,float64
3033384050822504448,2015.5,113.62449425983519,-13.013594013923822,0.27278518937869317
3033358448513212544,2015.5,113.49931101083143,-13.300183533655007,-0.2873760780375147
3033385180391581824,2015.5,113.52962059564396,-12.987524353361847,0.11156847147339947
3033359410594235904,2015.5,113.70304158773014,-13.360618132876574,-0.05316876795938308
3033396557768590720,2015.5,113.11768973380087,-13.302574978447408,0.8308880987589525
3033393254932819456,2015.5,112.8002316767924,-13.333955291946618,-0.23614303811919068
3033372162353266816,2015.5,113.48341800703552,-13.187552954799052,0.33449605907849567
3033388070913411072,2015.5,112.94026285237977,-13.422947057044333,0.3835557341558295


Asynchronous jobs have a `jobid`.

In [15]:
job1.jobid, job2.jobid

(None, '1595951250672O')

Which you can use to remove the job from the server.

In [16]:
Gaia.remove_jobs([job2.jobid])

Removed jobs: '['1595951250672O']'.


**Exercise:** The clauses in a query have to be in the right order.  Go back and change the order of the clauses in `query2` and run it again.  

They query should fail, but notice that you don't get much useful debugging information.  

For this reason, developing and debugging ADQL queries can be really hard.  A few suggestions that might help:

* Whenever possible, start with a working query, either an example you find online or a query you have used in the past.

* Make small changes and test each change before you continue.

* While you are debugging, use `TOP` to limit the number of rows in the result.  That will make each attempt run faster, which reduces your testing time.

**Exercise:**  In a `WHERE` clause, you can use any of the comparison operators:

* `>`: greater than
* `<`: less than
* `>=`: greater than or equal
* `<=`: less than or equal
* `=`: equal
* `<>`: not equal

Most of these are the same as Python, but some are not.  Be careful to keep your Python out of your ADQL!

You can combine comparisons using the logical operators:

* AND: true if both comparisons are true
* OR: true if either or both comparisons are true
* NOT: true 

[Read about SQL operators here](https://www.w3schools.com/sql/sql_operators.asp) and then modify the previous query to select rows where `bp_rp` is between `-0.75` and `2`.


## Formatting queries

So far the queries have been string "literals", meaning that the entire string is part of the program.
But writing queries yourself can be slow, repetitive, and error-prone.

It is often a good idea two write Python code that assembles a query for you.  One useful tool for that is the [string `format` method](https://www.w3schools.com/python/ref_string_format.asp).

As an example, here's a list of columns we might want to select.

In [40]:
columns = 'source_id, ra, dec, pmra, pmdec, parallax, parallax_error'

The following is a "base" for a query; it's a string that contains at least one format specifier in curly brackets (braces).

In [41]:
query3_base = """SELECT TOP 10
{columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1 AND 
bp_rp BETWEEN -0.75 AND 2
"""

This base query contains one format specifier, `{columns}`.

To assemble the query, we invoke `format` on the base string and provide a keyword arguments that assigns a value to `columns`.

In [42]:
query3 = query3_base.format(columns=columns)

We'll use the following function to print multi-line queries readably.

In [43]:
def print_query(query):
    """Print an ADQL query readably.
    
    query: string
    """
    for line in query.split('\n'):
        print(line)

Here's the query we just assembled.

In [44]:
print_query(query3)

SELECT TOP 10
source_id, ra, dec, pmra, pmdec, parallax, parallax_error
FROM gaiadr2.gaia_source
WHERE parallax < 1 AND 
bp_rp BETWEEN -0.75 AND 2



The format specifier has been replaced with the value of `names`.

Let's run it and see if it works:

In [22]:
job3 = Gaia.launch_job(query3)
print(job3)

<Table length=10>
     name       dtype    unit                              description                            
-------------- ------- -------- ------------------------------------------------------------------
     source_id   int64          Unique source identifier (unique within a particular Data Release)
            ra float64      deg                                                    Right ascension
           dec float64      deg                                                        Declination
          pmra float64 mas / yr                         Proper motion in right ascension direction
         pmdec float64 mas / yr                             Proper motion in declination direction
      parallax float64      mas                                                           Parallax
parallax_error float64      mas                                         Standard error of parallax
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20200728114734.xml.gz
Results: N

In [23]:
results3 = job3.get_results()
results3

source_id,ra,dec,pmra,pmdec,parallax,parallax_error
,deg,deg,mas / yr,mas / yr,mas,mas
int64,float64,float64,float64,float64,float64,float64
5983517509294909952,241.66142704493473,-48.78274641972377,-2.7567118969568303,-3.1683663439015506,0.6505088028742116,0.05877259987956226
5983525721272694272,241.096690737405,-49.18690961360837,-3.2766663311488813,-3.808637263240045,0.6649377407842838,0.046565042517965276
5352269501314712832,157.42254985703002,-56.27611942129379,-8.556963800672188,2.5408384185741957,-0.6937868475462103,0.7335461008689722
5352245140281179264,159.0729945730158,-55.21714452703379,-6.059848974283112,3.193406569975477,0.43155364920599126,0.47932408658697434
5352243250495343232,158.98934844993977,-55.29143493572026,-5.345033881738532,3.1154400584267257,-0.7847205976290007,0.5468582582897002
5352267577192177152,157.5906764026862,-56.29581702564557,-4.725619042190881,2.60243220809965,0.16656627525584938,0.08831005218482955
5352273482753457152,157.70249106942077,-56.20060724073244,-8.941450655100702,3.7316224903436064,0.7894879961800545,0.10746883978899523
5352272628073424768,157.42632985525785,-56.12458381376484,-15.704304461607071,-4.403260745449822,0.5800472179860736,0.03841710630537945


Good so far.

**Exercise:** This query always selects sources with `parallax` less than 1.  But suppose you want to take that upper bound as an input.

Modify `query3_base` to replace `1` with a format specifier like `{max_parallax}`.  Now, when you call `format`, add a keyword argument that assigns a value to `max_parallax`, and confirm that the format specifier gets replaced with the value you provide.

**Style note:**  You might notice that the variable names in this notebook are numbered, like `query1`, `query2`, etc.  

The advantage of this style is that it isolated each section of the notebook from the others, so if you go back and run the cells out of order, it's less likely that you will get unexpected interactions.

A drawback of this style is that it can be a nuisance to update the notebook if you add, remove, or reorder a section.

What do you think of this choice?  Are there alternatives you prefer?

## Selecting a region

One of the most common ways to restrict a query is to select stars in a particular region of the sky.

For example, here's a query from the [Gaia archive documentation](https://gea.esac.esa.int/archive-help/adql/examples/index.html) that selects "all the objects ... in a circular region centered at (266.41683, -29.00781) with a search radius of 5 arcmin (0.08333 deg)."

In [24]:
query = """
SELECT 
TOP 10 source_id
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
  POINT(ra, dec),
  CIRCLE(266.41683, -29.00781, 0.08333333))
"""

This query uses three keywords that are specific to ADQL (not SQL):

* `POINT`: a location in [ICRS coordinates](https://en.wikipedia.org/wiki/International_Celestial_Reference_System), specified in degrees of right ascension and declination.

* `CIRCLE`: a circle where the first two values are the coordinates of the center and the third is the radius in degrees.

* `CONTAINS`: a function that returns `1` if a `POINT` is contained in a shape and `0` otherwise.

Here is the [documentation of `CONTAINS`](http://www.ivoa.net/documents/ADQL/20180112/PR-ADQL-2.1-20180112.html#tth_sEc4.2.12).

A query like this is called a cone search because it selects stars in a cone.

In [25]:
job = Gaia.launch_job(query)
result = job.get_results()
result

source_id
int64
4057468321929794432
4057468287575835392
4057482027171038976
4057470349160630656
4057470039924301696
4057469868125641984
4057468351995073024
4057469661959554560
4057470520960672640


**Exercise:** When you are debugging queries like this, you can use `TOP` to limit the size of the results, but then you still don't know how big the results will be.

An alternative is to use `COUNT`, which asks for the number of rows that would be selected, but it does not return them.

In the previous query, replace `TOP 10 source_id` with `COUNT(source_id)` and run the query again.  How many stars has Gaia identified in the cone we searched?

## Getting GD-1 Data

From the Price-Whelan and Bonaca paper, we will try to reproduce Figure 1, which includes this representation of stars likely to belong to GD-1:

<img src="https://github.com/datacarpentry/astronomy-python/raw/gh-pages/fig/gd1-4.png">

Along the axis of right ascension ($\phi_1$) the figure extends from -100 to 20 degrees.

Along the axis of declination ($\phi_2$) the figure extends from about -8 to 4 degrees.

Ideally, we could select all stars from this rectangle, but there are more than 10 million of them, so

* That would be difficult to work with,

* As anonymous users, we are limited to 3 million rows in a single query, and

* While we are developing and testing code, it will be faster to work with a smaller dataset.

So I'll start by selecting stars in a smaller rectangle.

## Working with coordinates

Since coordinates are represented in degrees, I'll import `deg` from `astropy.units`.

In [26]:
import astropy.units as u

30 * u.deg

<Quantity 30. deg>

I'll select a rectangle from -55 to -45 degrees right ascension and -8 to 4 degrees of declination.

In [27]:
low1, high1 = -55, -45
phi1 = [low1, low1, high1, high1] * u.deg

low2, high2 = -8, 4
phi2 = [low2, high2, high2, low2] * u.deg

`phi1` and `phi2` represent the coordinates of the corners of a rectangle.  But they are in "[a Heliocentric spherical coordinate system defined by the orbit of the GD1 stream](https://gala-astro.readthedocs.io/en/latest/_modules/gala/coordinates/gd1.html)"

In order to use them in a Gaia query, we have to convert them to ICRS.  We can do that by storing the coordinates in a `GD1Koposov10` object provided by [Gala](https://gala-astro.readthedocs.io/en/latest/coordinates/).

In [28]:
import gala.coordinates as gc

corners = gc.GD1Koposov10(phi1=phi1, phi2=phi2)
type(corners)

gala.coordinates.gd1.GD1Koposov10

In [29]:
corners

<GD1Koposov10 Coordinate: (phi1, phi2) in deg
    [(-55., -8.), (-55.,  4.), (-45.,  4.), (-45., -8.)]>

Now we can use `transform_to` to convert to [International Celestial Reference System](https://en.wikipedia.org/wiki/International_Celestial_Reference_System) coordinates.

In [30]:
import astropy.coordinates as coord

corners_icrs = corners.transform_to(coord.ICRS)
type(corners_icrs)

astropy.coordinates.builtin_frames.icrs.ICRS

In [31]:
corners_icrs

<ICRS Coordinate: (ra, dec) in deg
    [(146.27533314, 19.26190982), (135.42163944, 25.87738723),
     (141.60264825, 34.3048303 ), (152.81671045, 27.13611254)]>

Notice that a rectangle in one coordinate system is not necessarily a rectangle in another.  In this example, the result is a polygon.

## Selecting a polygon

In order to use this polygon as part of an ADQL query, we have to convert it to a string with a comma-separated list of coordinates, as in this example:

```
"""
POLYGON(143.65740785846373, 20.98189112798802, 
        134.46717444171475, 26.39291776724364, 
        140.58825494277238, 34.85481376928442, 
        150.16628417989443, 29.015570791894923)
"""
```

`corners_icrs` behaves like a list, so we can use the bracket operator to select the first coordinate pair.

In [32]:
for point in corners_icrs:
    print(point)

<ICRS Coordinate: (ra, dec) in deg
    (146.27533314, 19.26190982)>
<ICRS Coordinate: (ra, dec) in deg
    (135.42163944, 25.87738723)>
<ICRS Coordinate: (ra, dec) in deg
    (141.60264825, 34.3048303)>
<ICRS Coordinate: (ra, dec) in deg
    (152.81671045, 27.13611254)>


From that, we can select the coordinates `ra` and `dec`:

In [33]:
for point in corners_icrs:
    print(point.ra, point.dec)

146d16m31.1993s 19d15m42.8754s
135d25m17.902s 25d52m38.594s
141d36m09.5337s 34d18m17.3891s
152d49m00.1576s 27d08m10.0051s


The results are in units of degrees, but if we select the `degrees` part, we get a dimensionless floating-point number.

In [34]:
for point in corners_icrs:
    print(point.ra.degree, point.dec.degree)

146.27533313607782 19.261909820533692
135.42163944306296 25.87738722767213
141.60264825107333 34.304830296257144
152.81671044675923 27.136112541397996


We can use string `format` to convert numbers to strings.

In [35]:
point_base = "{point.ra.degree}, {point.dec.degree}"

t = [point_base.format(point=point)
     for point in corners_icrs]
t

['146.27533313607782, 19.261909820533692',
 '135.42163944306296, 25.87738722767213',
 '141.60264825107333, 34.304830296257144',
 '152.81671044675923, 27.136112541397996']

The result is a list of strings.  We can use `join` to make a single string.

In [36]:
point_list = ', '.join(t)
point_list

'146.27533313607782, 19.261909820533692, 135.42163944306296, 25.87738722767213, 141.60264825107333, 34.304830296257144, 152.81671044675923, 27.136112541397996'

Finally, we can assemble the query.  Here's the base.

In [37]:
query4_base = """SELECT {columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1 AND bp_rp > -0.75 AND bp_rp < 2 AND
      1 = CONTAINS(POINT(ra, dec), 
                   POLYGON({point_list}))
"""

And here's the result:

In [45]:
query4 = query4_base.format(columns=columns, 
                            point_list=point_list)
print_query(query4)

SELECT source_id, ra, dec, pmra, pmdec, parallax, parallax_error
FROM gaiadr2.gaia_source
WHERE parallax < 1 AND bp_rp > -0.75 AND bp_rp < 2 AND
      1 = CONTAINS(POINT(ra, dec), 
                   POLYGON(146.27533313607782, 19.261909820533692, 135.42163944306296, 25.87738722767213, 141.60264825107333, 34.304830296257144, 152.81671044675923, 27.136112541397996))



Let's launch it.  The results will be bigger than our previous queries, so it will take a little longer.

In [46]:
job4 = Gaia.launch_job_async(query4)
print(job4)

INFO: Query finished. [astroquery.utils.tap.core]
<Table length=140340>
     name       dtype    unit                              description                            
-------------- ------- -------- ------------------------------------------------------------------
     source_id   int64          Unique source identifier (unique within a particular Data Release)
            ra float64      deg                                                    Right ascension
           dec float64      deg                                                        Declination
          pmra float64 mas / yr                         Proper motion in right ascension direction
         pmdec float64 mas / yr                             Proper motion in declination direction
      parallax float64      mas                                                           Parallax
parallax_error float64      mas                                         Standard error of parallax
Jobid: 1595951420248O
Phase: COMPLETE

Here are the results.

In [47]:
results4 = job4.get_results()
len(results4)

140340

There are more than 100,000 stars in this polygon, but that's a manageable size to work with.

## Saving results

This is the set of stars we'll work with in the next step.  But since we have a substantial dataset now, this is a good time to save it.

Storing the data in a file means we can shut down this notebook and pick up where we left off without running the previous query again.

Astropy `Table` objects provide `write`, which writes the table to disk.

In [48]:
filename = 'gd1_results4.fits'
results4.write(filename, overwrite=True)

Because the filename ends with `fits`, the table is written in the [FITS format](https://en.wikipedia.org/wiki/FITS), which preserves the metadata associated with the table.

If the file already exists, the `overwrite` argument causes it to be overwritten.

We can use the following function to see how big the file is.

In [49]:
import os

def filesize(filename):
    size = os.path.getsize(filename)
    print(size / 1024 / 1024, 'MB')

In [50]:
filesize(filename)

7.50640869140625 MB


We can read the file back like this:

In [52]:
from astropy.table import Table

filename = 'gd1_results4.fits'
results4 = Table.read(filename)

In [53]:
results4.info

<Table length=140340>
     name       dtype    unit                              description                            
-------------- ------- -------- ------------------------------------------------------------------
     source_id   int64          Unique source identifier (unique within a particular Data Release)
            ra float64      deg                                                    Right ascension
           dec float64      deg                                                        Declination
          pmra float64 mas / yr                         Proper motion in right ascension direction
         pmdec float64 mas / yr                             Proper motion in declination direction
      parallax float64      mas                                                           Parallax
parallax_error float64      mas                                         Standard error of parallax

## Making a function

In [54]:
def transform_rectangle(low1, high1, low2, high2):
    """
    """
    phi1 = [low1, low1, high1, high1] * u.deg
    phi2 = [low2, high2, high2, low2] * u.deg
    corners = gc.GD1Koposov10(phi1=phi1, phi2=phi2)
    corners_icrs = corners.transform_to(coord.ICRS)
    return corners_icrs

In [55]:
corners_icrs = transform_rectangle(-55, -45, -4, 6)

In [56]:
assert(corners_icrs[0].ra.degree == 142.7716385024318)

In [57]:
def make_point_list(coords):
    """
    """
    point_base = "{point.ra.degree}, {point.dec.degree}"

    t = [point_base.format(point=point)
         for point in coords]

    poly_base = "POLYGON({point_list})"
    point_list = ', '.join(t)
    return point_list

In [58]:
corners_icrs = transform_rectangle(-55, -45, -5, 5)
point_list = make_point_list(corners_icrs)
point_list

'143.65740785846373, 20.98189112798802, 134.46717444171475, 26.39291776724364, 140.58825494277238, 34.85481376928442, 150.16628417989443, 29.015570791894923'